In [1]:
import os, pickle
import numpy as np
import pandas as pd
from scipy import stats

FEATURES_BASE_PATH: str = "../input/fic-swin-features"
MODELS_BASE_PATH: str = "../input/fic-swin-features-train/models"
MODEL_NAMES: str = os.listdir(MODELS_BASE_PATH)

In [2]:
ts_base_features  = np.load("../input/fic-swin-features/ts_swin_base_patch4_window12_384_in22k.npy")
ts_large_features = np.load("../input/fic-swin-features/ts_swin_large_patch4_window12_384_in22k.npy")


y_pred = np.zeros((1, ts_base_features.shape[0]), dtype=np.uint8)

for model_name in MODEL_NAMES:
    if "lgr" in model_name or "knc" in model_name or "svc" in model_name or "rfc" in model_name or "etcs" in model_name:
        if "base" in model_name:
            model = pickle.load(open(os.path.join(MODELS_BASE_PATH, model_name), "rb"))
            y_pred = np.concatenate((y_pred, model.predict(ts_base_features).reshape(1, -1)), axis=0)
        if "large" in model_name:
            model = pickle.load(open(os.path.join(MODELS_BASE_PATH, model_name), "rb"))
            y_pred = np.concatenate((y_pred, model.predict(ts_large_features).reshape(1, -1)), axis=0)

y_pred = stats.mode(y_pred[1:])[0]
                                    
ss_df = pd.read_csv("../input/5-flowers-image-classification/Sample_submission.csv")
ss_df.label = y_pred.astype("uint8").reshape(-1)
ss_df.to_csv("submission.csv", index=False)